# Dependencies

In [1]:
import pandas as pd
import pymongo
from pprint import pprint

# Import

In [2]:
#timeline information
search_apex = pd.read_csv('Resources/multiTimeline(Apex).csv')
search_csgo = pd.read_csv('Resources/multiTimeline(CSGO).csv')
search_dota = pd.read_csv('Resources/multiTimeline(DOTA).csv')
# search_nw = pd.read_csv('Resources/multiTimeline(NewWorld).csv')
search_rust = pd.read_csv('Resources/multiTimeline(Rust).csv')
search_gta = pd.read_csv('Resources/multiTimeline(GTAV).csv')

#geomap information
geo_apex = pd.read_csv('Resources/geoMap(Apex).csv')
geo_csgo = pd.read_csv('Resources/geoMap(CSGO).csv')
geo_dota = pd.read_csv('Resources/geoMap(DOTA).csv')
# geo_nw = pd.read_csv('Resources/geoMap(NewWorld).csv')
geo_rust = pd.read_csv('Resources/geoMap(Rust).csv')
geo_gta = pd.read_csv('Resources/geoMap(GTAV).csv')

# Clean and Format

In [3]:
search_ls = [search_apex, search_csgo, search_dota, search_gta, search_rust]
merged_search = search_ls[0].merge(search_ls[1], on = 'Month', how = 'outer')
for i in range(2, len(search_ls)):
    merged_search = merged_search.merge(search_ls[i], on = 'Month', how = 'outer')
merged_search.rename(columns = {merged_search.columns[1] : 'Apex',
                                merged_search.columns[2] : 'CSGO',
                                merged_search.columns[3] : 'Dota 2',
                                merged_search.columns[4] : 'GTA V',
                                merged_search.columns[5] : 'Rust'
}, inplace = True)
merged_search.replace('<1', 1, inplace = True)
merged_search = merged_search.astype({'Apex' : 'int64', 'Dota 2': 'int64'})
search_dict = merged_search.to_dict('records')

In [4]:
geo_ls = [geo_apex, geo_csgo, geo_dota, geo_gta, geo_rust]
merged_geo = geo_ls[0].merge(geo_ls[1], on = 'Region', how = 'outer')
for i in range(2, len(geo_ls)):
    merged_geo = merged_geo.merge(geo_ls[i], on = 'Region', how = 'outer')
merged_geo.rename(columns = {merged_geo.columns[1] : 'Apex',
                             merged_geo.columns[2] : 'CSGO',
                             merged_geo.columns[3] : 'Dota 2',
                             merged_geo.columns[4] : 'GTA V',
                             merged_geo.columns[5] : 'Rust'
}, inplace = True)
merged_geo.sort_values('Region', inplace = True)
merged_geo.reset_index(drop = True, inplace = True)
geo_dict = merged_geo.to_dict('records')

In [5]:
# search_dict
# geo_dict

# Export to MongoDB

In [6]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.googletrends_db

# declare the collection
g_line = db.multiline
g_geo = db.geo

In [7]:
# check databases
print(client.list_database_names())

['Dumpster_DB', 'admin', 'config', 'fruits_db', 'local', 'store_inventory', 'trending_games_db']


In [8]:
# insert data
g_line.insert_many(search_dict)
g_geo.insert_many(geo_dict)

In [9]:
# check/read data
read_result_line = g_line.find_one()
pprint(read_result_line)

read_result_geo = g_geo.find_one()
pprint(read_result_geo)

{'Apex': 0,
 'CSGO': 58,
 'Dota 2': 0,
 'GTA V': 1,
 'Month': '2004-01',
 'Rust': 23,
 '_id': ObjectId('61e2ef742cdad25db0d36180')}
{'Apex': 66,
 'CSGO': 54,
 'Dota 2': 40,
 'GTA V': 74,
 'Region': 'Alabama',
 'Rust': 66,
 '_id': ObjectId('61e2ef742cdad25db0d36259')}
